# NGAME training pipeline with clustering

In [2]:
#| default_exp 16-ngame-training-pipeline-with-clustering-3-0

In [3]:
#| hide
from nbdev.showdoc import *
import nbdev; nbdev.nbdev_export()

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
#| export
import os,torch, torch.multiprocessing as mp
from xcai.basics import *
from xcai.models.MMM00X import DBT012

In [6]:
os.environ['WANDB_MODE'] = 'disabled'

In [5]:
#| export
os.environ['WANDB_PROJECT']='xc-nlg_16-ngame-training-pipeline-with-clustering-3'

In [6]:
#| export
block = XCBlock.from_cfg('/home/aiscuser/scratch/datasets', 'data', valid_pct=0.001, tfm='ng', 
                         tokenizer='sentence-transformers/msmarco-distilbert-base-v4')

/opt/conda/envs/ptca/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/conda/envs/ptca/lib/python3.9/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [7]:
#| export
args = XCLearningArguments(
    output_dir='/home/aiscuser/outputs/16-ngame-training-pipeline-with-clustering-3-0',
    logging_first_step=True,
    per_device_train_batch_size=1024,
    per_device_eval_batch_size=64,
    representation_num_beams=200,
    representation_accumulation_steps=100,
    save_strategy="steps",
    evaluation_strategy='steps',
    eval_steps=1000,
    save_steps=1000,
    save_total_limit=5,
    num_train_epochs=50,
    predict_with_representation=True,
    label_names=['plbl2data_idx', 'plbl2data_data2ptr'],
    target_indices_key='plbl2data_idx',
    target_pointer_key='plbl2data_data2ptr',
    adam_epsilon=1e-6,
    warmup_steps=100,
    weight_decay=0.01,
    learning_rate=2e-4,
    group_by_cluster=True,
    num_clustering_warmup_epochs=10,
    num_cluster_update_epochs=5,
    num_cluster_size_update_epochs=5,
    clustering_type='EXPO',
    minimum_cluster_size=1,
    maximum_cluster_size=None,
)

In [8]:
#| export
metric = PrecRecl(block.n_lbl, block.test.data_lbl_filterer, prop=block.train.dset.data.data_lbl,
                  pk=10, rk=200, rep_pk=[1, 3, 5, 10], rep_rk=[10, 100, 200])

In [10]:
#| export
model = DBT012.from_pretrained('sentence-transformers/msmarco-distilbert-base-v4', margin=0.3, tau=0.1, 
                               apply_softmax=True, n_negatives=5)

/opt/conda/envs/ptca/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [10]:
#| export
learn = XCLearner(
    model=model, 
    args=args,
    train_dataset=block.train.dset,
    eval_dataset=block.test.dset,
    data_collator=block.collator,
    compute_metrics=metric,
    
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
learn.train()

In [ ]:
#| export
if __name__ == '__main__':
    mp.freeze_support()
    learn.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/5410 [00:00<?, ?it/s]

Updating clusters with size 1
Tree depth = 20
doing random split
lengths: [346194, 346195]
remaining levels for GPU split=19


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


==> gpu splitting random clusters 0 to 2


...

Updating clusters with size 2
Tree depth = 19
doing random split
lengths: [346194, 346195]
remaining levels for GPU split=18


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


==> gpu splitting random clusters 0 to 2


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 rank=1 => Total clusters 2	Avg. Cluster size                 173097.50	Time to split nodes on this level 0.97 sec
 rank=1 => Total clusters 4	Avg. Cluster size                 86548.75	Time to split nodes on this level 0.06 sec
 rank=0 => Total clusters 2	Avg. Cluster size                 173097.00	Time to split nodes on this level 0.88 sec
 rank=0 => Total clusters 4	Avg. Cluster size                 86548.50	Time to split nodes on this level 0.04 sec
 rank=1 => Total clusters 8	Avg. Cluster size                 43274.38	Time to split nodes on this level 0.08 sec
 rank=0 => Total clusters 8	Avg. Cluster size                 43274.25	Time to split nodes on this level 0.07 sec
 rank=1 => Total clusters 16	Avg. Cluster size                 21637.19	Time to split nodes on this level 0.08 sec
 rank=0 => Total clusters 16	Avg. Cluster size                 21637.12	Time to split nodes on this level 0.07 sec
 rank=1 => Total clusters 32	Avg. Cluster size                 10818.59	Time to spli

In [ ]:
%debug

In [2]:
import torch

In [31]:
a = torch.tensor([0, 0, 0, 0])

In [20]:
x = torch.where(a == 0, -1e9, a)

In [30]:
x

tensor([-1.0000e+09, -1.0000e+09,  1.0000e-01, -1.0000e+09])

In [29]:
torch.nn.functional.softmax(x, dim=0)/(torch.tensor([1, 0, 1, 1])+1e-9)

tensor([0., 0., 1., 0.])

tensor([1, 0, 1, 1])